div#notebook p, div#notebook{
    font-size: 130%;
    line-height: 125%;
}
.rendered_html pre, .rendered_html table{
    font-size:130%;
    line-height: 125%;
}
.CodeMirror, .CodeMirror pre, .CodeMirror-dialog, .CodeMirror-dialog .CodeMirror-search-field, .terminal-app .terminal {
    font-size: 130%;
    line-height: 125%;
}

<div style=style="font-size: 20px">
   
     your stuffs here

</div>

!jt -t oceans16 -tf merriserif -tfs 14 -nf ptsans -nfs 14
### Reference: https://gist.github.com/endolith/334196bac1cac45a4893

##### Automatically detect rotation and line spacing of an image of text using
##### Radon transform

##### If image is rotated by the inverse of the output, the lines will be
##### horizontal (though they may be upside-down depending on the original image)

##### It doesn't work with black borders

<font size="5">dadasda
asdasdsa
asdasd</font>


In [1]:
from __future__ import division, print_function
from skimage.transform import radon
from PIL import Image
from PIL.Image import Resampling
from numpy import asarray, mean, array, blackman
import numpy
from numpy.fft import rfft
import matplotlib.pyplot as plt

# from matplotlib.mlab import rms_flat
from matplotlib import mlab
import cv2

try:
    # More accurate peak finding from
    # https://gist.github.com/endolith/255291#file-parabolic-py
    from parabolic import parabolic

    def argmax(x):
        return parabolic(x, numpy.argmax(x))[0]

except ImportError:
    from numpy import argmax

In [ ]:
name = "text_" + "right"
path_before = f"project/data/photos/rotate_text/after_remove_shadow/{name}.jpg"
# path_after = f"photos/rotate_text/after_rotated/{name}.jpg"

# Load file, converting to grayscale
I = asarray(Image.open(path_before).convert("L"))

In [ ]:
name = "text_" + "right"
path_before = f"photos/rotate_text/after_remove_shadow/{name}.jpg"
path_after = f"photos/rotate_text/after_rotated/{name}.jpg"

# Load file, converting to grayscale
I = asarray(Image.open(path_before).convert("L"))
I = I - mean(I)  # Demean; make the brightness extend above and below zero
plt.subplot(2, 2, 1)
plt.imshow(I)

# Do the radon transform and display the result
sinogram = radon(I)

plt.subplot(2, 2, 2)
plt.imshow(sinogram.T, aspect="auto")
plt.gray()

# Find the RMS value of each row and find "busiest" rotation,
# where the transform is lined up perfectly with the alternating dark
# text and white lines
rms = lambda x_seq: (sum(x * x for x in x_seq) / len(x_seq)) ** (1 / 2)
# r = array([lambda x_seq: (sum(x*x for x in x_seq)/len(x_seq))**(1/2) for line in sinogram.transpose()])
r = array([rms(line) for line in sinogram.transpose()])
rotation = argmax(r)
print("Rotation: {:.2f} degrees".format(90 - rotation))
plt.axhline(rotation, color="r")

# Plot the busy row
row = sinogram[:, rotation]
N = len(row)
plt.subplot(2, 2, 3)
plt.plot(row)

# Take spectrum of busy row and find line spacing
window = blackman(N)
spectrum = rfft(row * window)
plt.plot(row * window)
frequency = argmax(abs(spectrum))
line_spacing = N / frequency  # pixels
print("Line spacing: {:.2f} pixels".format(line_spacing))

plt.subplot(2, 2, 4)
plt.plot(abs(spectrum))
plt.axvline(frequency, color="r")
plt.yscale("log")
plt.show()

In [ ]:
# rotating a image 90 deg counter clockwise
rot = rotation - 90
im1 = Image.open(path_before)
# im1 = im1.rotate(rot, Resampling.NEAREST , expand = 1)
im1 = im1.rotate(rot)

# to show specified image
# plt.get_frame()
plt.axis("off")
# plt.ax.set_frame_on(False)
plt.xticks([])
plt.yticks([])
plt.imshow(im1)
plt.savefig(path_after, bbox_inches="tight")